In [37]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from datetime import datetime, timedelta  # Import necessary modules


In [16]:
df = yf.download(tickers = 'BTC-USD', start = '2019-04-15',end = str(pd.Timestamp.today()).split(' ')[0])
df = df.drop(columns=['Open','High','Low','Close','Volume'])
df = df.dropna()

[*********************100%%**********************]  1 of 1 completed


In [17]:
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df)

In [18]:
sequence_length = 10
X, y = [], []

In [19]:
for i in range(len(df_scaled) - sequence_length):
    X.append(df_scaled[i:i+sequence_length, 0])
    y.append(df_scaled[i+sequence_length, 0])

In [20]:
X, y = np.array(X), np.array(y)

In [21]:
X = np.reshape(X, (X.shape[0], X.shape[1], 1))


In [23]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [31]:
model.fit(X, y, epochs=20, batch_size=32)

Epoch 1/20


56/56 [==============================] - 0s 5ms/step - loss: 6.1682e-04
Epoch 2/20
56/56 [==============================] - 0s 4ms/step - loss: 6.3523e-04
Epoch 3/20
56/56 [==============================] - 0s 4ms/step - loss: 6.1871e-04
Epoch 4/20
56/56 [==============================] - 0s 4ms/step - loss: 7.1116e-04
Epoch 5/20
56/56 [==============================] - 0s 4ms/step - loss: 7.0539e-04
Epoch 6/20
56/56 [==============================] - 0s 4ms/step - loss: 6.3372e-04
Epoch 7/20
56/56 [==============================] - 0s 4ms/step - loss: 6.4382e-04
Epoch 8/20
56/56 [==============================] - 0s 5ms/step - loss: 5.8075e-04
Epoch 9/20
56/56 [==============================] - 0s 4ms/step - loss: 5.9550e-04
Epoch 10/20
56/56 [==============================] - 0s 4ms/step - loss: 5.3438e-04
Epoch 11/20
56/56 [==============================] - 0s 4ms/step - loss: 5.0340e-04
Epoch 12/20
56/56 [==============================] - 0s 4ms/step - loss: 5.0242e-04
Epoch 13/20


In [33]:
last_sequence = df_scaled[-sequence_length:].reshape((1, sequence_length, 1))
future_predictions = []

In [34]:
for i in range(7):
    next_day_prediction = model.predict(last_sequence)
    future_predictions.append(next_day_prediction[0, 0])
    last_sequence = np.roll(last_sequence, -1, axis=1)
    last_sequence[0, -1, 0] = next_day_prediction[0, 0]

1/1 [==============================] - 0s 14ms/step


In [35]:
future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))


In [38]:
tomorrow_date = datetime.now() + timedelta(days=1)

In [39]:
print("Predicted Prices for the Next 7 Days:")
for i in range(7):
    print(f"{tomorrow_date.strftime('%Y-%m-%d')}: {future_predictions[i][0]}")
    tomorrow_date += timedelta(days=1)

Predicted Prices for the Next 7 Days:
2024-03-12: 68126.296875
2024-03-13: 67789.8203125
2024-03-14: 67196.65625
2024-03-15: 66495.9375
2024-03-16: 65705.7109375
2024-03-17: 64938.765625
2024-03-18: 64164.69140625


In [40]:
#dfada